In [ ]:
from datetime import datetime
import pandas as pd
import seaborn as sns

In [ ]:
classification_df = pd.read_csv('demand_classification_by_item.csv', sep=';')
raw_df = pd.read_csv('df_periodos_rellenados.csv', sep=';')
display(raw_df)
display(classification_df)

In [ ]:
raw_df.time.min()

In [ ]:
raw_df['time'] = pd.to_datetime(raw_df['time'], format="%Y-%m-%d")
raw_df['day'] = raw_df.time.dt.day
raw_df['month'] = raw_df.time.dt.month
raw_df['year'] = raw_df.time.dt.year
raw_df['weekday'] = raw_df.time.dt.weekday
raw_df['is_weekend'] = (raw_df.weekday >= 5).astype(int)
raw_df['days_since_first_data'] = (raw_df.time - raw_df.time.min()).dt.days
raw_df

### Solo consideramos los ítems de demanda Lumpy e Intermittent

In [ ]:
intermittent_ids = classification_df[classification_df.demand_type == 'intermittent']['item'].unique()
lumpy_ids = classification_df[classification_df.demand_type == 'lumpy']['item'].unique()
raw_df = raw_df[(raw_df['item'].isin(intermittent_ids)) | (raw_df['item'].isin(lumpy_ids))]
df = raw_df[['item', 'day', 'month', 'year', 'weekday', 'is_weekend', 'days_since_first_data', 'sales']]
df

### Dividimos en set de entrenamiento, test y validación

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from darts import TimeSeries
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.seasonal import seasonal_decompose

from sklearn.metrics import mean_squared_error, mean_absolute_error

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from statsmodels.stats.diagnostic import acorr_ljungbox
import pmdarima as pm
from darts.models import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df, test_df = train_test_split(df, train_size=0.8)

In [ ]:
X_train = train_df[['item', 'day', 'month', 'year', 'weekday', 'is_weekend', 'days_since_first_data']]
X_test = test_df[['item', 'day', 'month', 'year', 'weekday', 'is_weekend', 'days_since_first_data']]
y_train = train_df['sales']
y_test = test_df['sales']

In [ ]:
print(f'Tamaño de entrenamiento: {len(X_train)}')
print(f'Tamaño de prueba: {len(X_test)}')

In [ ]:
scaler = StandardScaler()

# Escalar características de entrenamiento y prueba
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
test_df['y_pred'] = y_pred
test_df

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i] != 0.0 or y_test.values[i] != 0:
        print(f'y_pred: {y_pred[i]} | y_test: {y_test.values[i]}')

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mae

In [ ]:
"""
from sklearn.model_selection import GridSearchCV

# Definir los parámetros a ajustar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizar búsqueda en malla con validación cruzada
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Ver los mejores parámetros
print(grid_search.best_params_)
"""

### XGBRegressor

In [ ]:
import xgboost as xgb

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model_xgb.fit(X_train_scaled, y_train)

In [ ]:
y_pred_xgb = model_xgb.predict(X_test_scaled)
y_pred_xgb

In [ ]:
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mae_xgb

In [ ]:
test_df['y_pred_xgb'] = y_pred_xgb

In [ ]:
len(test_df.item.unique())

In [ ]:
test_df['y_pred'] = test_df['y_pred'].apply(lambda x: round(x))
test_df['y_pred_xgb'] = test_df['y_pred_xgb'].apply(lambda x: round(x))

In [ ]:
test_df.head()

In [ ]:
mae_rf = mean_absolute_error(test_df['sales'], test_df['y_pred'])
mae_rf

In [ ]:
mae_xgb = mean_absolute_error(test_df['sales'], test_df['y_pred_xgb'])
mae_xgb

## Desarrollo de métrica de utilidad

In [ ]:
len(classification_df.item.unique())

In [ ]:
classification_df

In [ ]:
test_df = test_df.reset_index(drop=True)
test_df

In [ ]:
classification_dict = classification_df.to_dict()
item_price = {}
for i in classification_dict['unit_price'].keys():
    item_price[classification_dict['item'][i]] = classification_dict['unit_price'][i]

item_price

In [ ]:
# Agregamos el precio unitario a cada item
test_df['unit_price'] = test_df['item'].apply(lambda x: item_price[x])
test_df

In [ ]:
utility_df = test_df.copy()[['item', 'sales', 'y_pred', 'y_pred_xgb', 'unit_price']]
utility_df

In [ ]:
# Precio promedio productos
classification_df['unit_price'].mean() 

In [ ]:
# Definimos un costo fijo de inventario para todos los productos
# Este costo lo definimos como un 30% del precio promedio de todos los productos
STOCK_COST = classification_df['unit_price'].mean() * 0.3
STOCK_COST

In [ ]:
# Calculamos los costos por exceso de inventario por cada modelo
def get_stock_cost(row, model):
    """
    Calcula el costo de inventario cuando la cantidad predicha es mayor o igual a la cantidad observada,
    según cada modelo.
    """
    if model == 'rf':
        target = 'y_pred'
    elif model == 'xgb':
        target = 'y_pred_xgb'
    else:
        target = 'y_pred'


    if row[target] >= row['sales']:
        stock_in_excess = (row[target] - row['sales']) * STOCK_COST
    else:
        stock_in_excess = 0

    return stock_in_excess

for m in ['rf', 'xgb']:
    utility_df[f'excess_stock_cost_{m}'] = utility_df.apply(lambda x: get_stock_cost(x, m), axis=1)

utility_df

In [ ]:
# Calculamos los costos por quiebre de stock por cada modelo
def get_stock_out_cost(row, model):
    """
    Calcula el costo de quiebre de stock cuando la cantidad predicha es menor a la cantidad observada,
    según cada modelo. Este costo está dado por el precio de venta del item por la cantidad de ítems no vendidos 
    a causa del quiebre de stock
    """
    if model == 'rf':
        target = 'y_pred'
    elif model == 'xgb':
        target = 'y_pred_xgb'
    else:
        target = 'y_pred'


    if row['sales'] > row[target]:
        stock_out = (row['sales'] - row[target]) * row['unit_price']
    else:
        stock_out = 0

    return stock_out

for m in ['rf', 'xgb']:
    utility_df[f'stock_out_cost_{m}'] = utility_df.apply(lambda x: get_stock_out_cost(x, m), axis=1)

utility_df

In [ ]:
# Calculamos los ingresos por venta de repuesto por cada modelo
def get_income_earned_by_sale(row, model):
    """
    Calcula el ingreso obtenido por la venta de repuesto
    """
    if model == 'rf':
        target = 'y_pred'
    elif model == 'xgb':
        target = 'y_pred_xgb'
    else:
        target = 'y_pred'


    if row[target] >= row['sales']:
        sales = row['sales'] * row['unit_price']
    else:
        sales = row[target] * row['unit_price']

    return sales

for m in ['rf', 'xgb']:
    utility_df[f'sales_income_{m}'] = utility_df.apply(lambda x: get_income_earned_by_sale(x, m), axis=1)

utility_df

In [ ]:
# Calculamos la utilidad para cada dato
def get_utility(row, model):
    """
    Calculamos la utilidad para cada dato:
    Suma de ingresos - Suma de costos
    """
    total_incomes = row[f'sales_income_{model}']
    total_costs = row[f'excess_stock_cost_{model}'] + row[f'stock_out_cost_{model}']
    utility = total_incomes - total_costs

    return utility

for m in ['rf', 'xgb']:
    utility_df[f'utility_{m}'] = utility_df.apply(lambda x: get_utility(x, m), axis=1)

utility_df

In [ ]:
# Calculamos la utilidad final para cada modelo
utility = {}
for m in ['rf', 'xgb']:
    utility[m] = utility_df[f'utility_{m}'].sum()

utility

### Transformers de series de tiempo

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
raw_df[raw_df.sales == 0]

In [ ]:
hf_df = raw_df[['time', 'item', 'sales']]
pivot_df = pd.pivot_table(hf_df, index='time', columns=['item'])
pivot_df = pivot_df['sales']

In [ ]:
items = pivot_df.columns

In [ ]:
pivot_df

In [ ]:
# Función para crear secuencias de datos
def crear_secuencias(datos, n_dias):
    X, y = [], []
    for i in range(len(datos) - n_dias):
        X.append(datos[i:i+n_dias])  # Datos históricos (ventas anteriores)
        y.append(datos[i + n_dias])  # El siguiente valor (ventas del día siguiente)
    return X, y

In [ ]:
items.difference([])

In [ ]:
# Definir el número de días para la secuencia
n_dias = 30

# Crear las secuencias para cada ítem
secuencias = {}
for item in items:
    item_serie = pivot_df[item].dropna()
    secuencias[item] = crear_secuencias(item_serie, n_dias)

# Dividir en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = {}, {}, {}, {}

discarded_items = []
for item in items:
    if len(secuencias[item][0]) > 0 and len(secuencias[item][1]):
        X_item, y_item = secuencias[item]
        X_train[item], X_test[item], y_train[item], y_test[item] = train_test_split(X_item, y_item, test_size=0.2, shuffle=False)
    else:
        discarded_items.append(item)

items = items.difference(discarded_items)
# Ver el tamaño de los datos
for item in items:
    print(f"{item} - Entrenamiento: {len(X_train[item])}, Prueba: {len(X_test[item])}")


In [ ]:
len(X_test.keys())

In [ ]:
scalers = {}
X_train_scaled, X_test_scaled = {}, {}

# Escalar los datos de ventas por ítem
for item in items:
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled[item] = scaler.fit_transform(X_train[item])
    X_test_scaled[item] = scaler.transform(X_test[item])
    scalers[item] = scaler


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)  # La salida de la LSTM
        out = self.fc(out[:, -1, :])  # Solo tomamos la última salida de la secuencia
        return out

In [ ]:
# Entrenamiento del modelo para cada ítem
models = {}
criterions = {}
optimizers = {}

for item in items:
    print(f'Item: {item}')
    # Convertir los datos a tensores
    X_train_tensor = torch.tensor(X_train_scaled[item], dtype=torch.float32).unsqueeze(-1)
    y_train_tensor = torch.tensor(y_train[item], dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test_scaled[item], dtype=torch.float32).unsqueeze(-1)
    y_test_tensor = torch.tensor(y_test[item], dtype=torch.float32)

    # Inicializar el modelo, la función de pérdida y el optimizador
    model = LSTMModel(input_size=1, hidden_layer_size=50, output_size=1)
    criterion = nn.MSELoss()  # Usamos error cuadrático medio para regresión
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    models[item] = model
    criterions[item] = criterion
    optimizers[item] = optimizer

    # Entrenamiento del modelo
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs.squeeze(), y_train_tensor)
        loss.backward()
        optimizer.step()

        # if (epoch+1) % 5 == 0:
        #     print(f"Item: {item}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


In [ ]:
# Hacer predicciones y evaluar el rendimiento
predictions = {}
for item in items:
    model = models[item]
    X_test_tensor = torch.tensor(X_test_scaled[item], dtype=torch.float32).unsqueeze(-1)
    y_test_tensor = torch.tensor(y_test[item], dtype=torch.float32)

    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)

    # Convertir predicciones y valores reales a numpy
    y_pred = y_pred_tensor.squeeze().numpy()
    y_test_actual = y_test_tensor.numpy()

    # Calcular el error absoluto medio (MAE)
    mae = mean_absolute_error(y_test_actual, y_pred)
    print(f"Item: {item}, MAE: {mae:.2f}")

    predictions[item] = {
        'mae': mae,
        'y_pred': y_pred,
        'y_test': y_test_actual
    }

    # Graficar las predicciones vs las ventas reales
    # plt.figure(figsize=(10, 6))
    # plt.plot(df.index[-len(y_test_actual):], y_test_actual, label='Ventas reales', color='blue')
    # plt.plot(df.index[-len(y_pred):], y_pred, label='Predicciones', color='red', linestyle='--')
    # plt.legend()
    # plt.title(f'Predicción de ventas con LSTM - {item}')
    # plt.xlabel('Fecha')
    # plt.ylabel('Ventas')
    # plt.xticks(rotation=45)
    # plt.show()


In [ ]:
X_test

In [ ]:
maes = [predictions[p]['mae'] for p in predictions]
np.mean(maes)

In [ ]:
test_items = list(predictions.keys())
len(test_items)

In [ ]:
len(predictions)

In [ ]:
secuencias

In [ ]:
predictions

In [ ]:
predictions[5432]['y_pred']
predictions[5432]['y_test']

In [ ]:
test_df['y_pred'] = test_df['y_pred'].apply(lambda x: round(x))
test_df['y_pred_xgb'] = test_df['y_pred_xgb'].apply(lambda x: round(x))

test_df

In [ ]:
test_df['y_pred_lstm'] = y_pred